In [73]:
import pandas as pd
from nltk.corpus import movie_reviews
from  sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDClassifier
from sklearn.cross_validation import cross_val_score
import pickle
from nltk.corpus import stopwords
words = stopwords.words('english')

# Изе скрин

In [80]:
train = pd.read_csv('/Users/alexey.tyshko/Desktop/all/products_sentiment_train.tsv', sep = '\t', names = ['Text', 'Target'])
test = pd.read_csv('/Users/alexey.tyshko/Desktop/all/products_sentiment_test.tsv', sep = '\t', names = ['Text'])

In [130]:
train.shape

(2000, 2)

In [135]:
test.Text[1]

"so , why the small digital elph , rather than one of the other cameras with better resolution or picture quality ? size [ + 2 ] # # because , unless it 's small , i won 't cary it around ."

# Model

Лог регрессия со ст. параметрами

Подбор параметров для векторайзера

In [50]:
score_1 = []
for min_df in (1, 2, 10, 20, 30, 40, 50):
    pipeline_1 = Pipeline([("vectorizer", CountVectorizer(min_df=min_df)), ("Regression", LogisticRegression())])
    score_1.append((cross_val_score(pipeline_1, train.Text, train.Target, scoring='accuracy', cv=5).mean(), min_df))
print "CountVectorizer: ", max(score_1, key=lambda x: x[0])

CountVectorizer:  (0.7699844436527729, 2)


In [51]:
score_2 = []
for min_df in (1, 2, 3, 5, 7, 9, 11):
    pipeline_1 = Pipeline([("vectorizer", TfidfVectorizer(min_df=min_df)), ("Regression", LogisticRegression())])
    score_2.append((cross_val_score(pipeline_1, train.Text, train.Target, scoring='accuracy', cv=5).mean(), min_df))
print "TfidfVectorizer:", max(score_2, key=lambda x: x[0])

TfidfVectorizer: (0.7665031843949025, 1)


Попробуем добавить стопслова

In [52]:
pipeline_words_1 = Pipeline([("vectorizer", CountVectorizer(min_df=2, stop_words = 'english')), ("Regression", LogisticRegression())])
print cross_val_score(pipeline_words_1, train.Text, train.Target, scoring='accuracy', cv=5).mean()
pipeline_words_2 = Pipeline([("vectorizer", CountVectorizer(min_df=2, stop_words = words)), ("Regression", LogisticRegression())])
print cross_val_score(pipeline_words_2, train.Text, train.Target, scoring='accuracy', cv=5).mean()

0.7459943687148044
0.750003121894512


In [53]:
pipeline_words_1 = Pipeline([("vectorizer", TfidfVectorizer(min_df=1, stop_words = 'english')), ("Regression", LogisticRegression())])
print cross_val_score(pipeline_words_1, train.Text, train.Target, scoring='accuracy', cv=5).mean()
pipeline_words_2 = Pipeline([("vectorizer", TfidfVectorizer(min_df=1, stop_words = words)), ("Regression", LogisticRegression())])
print cross_val_score(pipeline_words_2, train.Text, train.Target, scoring='accuracy', cv=5).mean()

0.7394981218632617
0.7480068937930862


фигня короче полная

Попробуем SVC

In [54]:
score_1 = []
for min_df in (1, 2, 10, 20, 30, 40, 50):
    pipeline_1 = Pipeline([("vectorizer", CountVectorizer(min_df=min_df)), ("Regression", LinearSVC())])
    score_1.append((cross_val_score(pipeline_1, train.Text, train.Target, scoring='accuracy', cv=5).mean(), min_df))
print "CountVectorizer: ", max(score_1, key=lambda x: x[0])

CountVectorizer:  (0.754000653129082, 1)


In [55]:
score_2 = []
for min_df in (1, 2, 3, 5, 7, 9, 11):
    pipeline_1 = Pipeline([("vectorizer", TfidfVectorizer(min_df=min_df)), ("Regression", LinearSVC())])
    score_2.append((cross_val_score(pipeline_1, train.Text, train.Target, scoring='accuracy', cv=5).mean(), min_df))
print "TfidfVectorizer:", max(score_2, key=lambda x: x[0])

TfidfVectorizer: (0.7734907093169333, 2)


Добавим стопслова

In [58]:
pipeline_words_1 = Pipeline([("vectorizer", CountVectorizer(min_df=1, stop_words = 'english')), ("Regression", LinearSVC())])
print cross_val_score(pipeline_words_1, train.Text, train.Target, scoring='accuracy', cv=5).mean()
pipeline_words_2 = Pipeline([("vectorizer", CountVectorizer(min_df=1, stop_words = words)), ("Regression", LinearSVC())])
print cross_val_score(pipeline_words_2, train.Text, train.Target, scoring='accuracy', cv=5).mean()

0.735988109300683
0.7380043562772267


In [59]:
pipeline_words_1 = Pipeline([("vectorizer", TfidfVectorizer(min_df=2, stop_words = 'english')), ("Regression", LinearSVC())])
print cross_val_score(pipeline_words_1, train.Text, train.Target, scoring='accuracy', cv=5).mean()
pipeline_words_2 = Pipeline([("vectorizer", TfidfVectorizer(min_df=2, stop_words = words)), ("Regression", LinearSVC())])
print cross_val_score(pipeline_words_2, train.Text, train.Target, scoring='accuracy', cv=5).mean()

0.7334843749023432
0.7404906218163864


Короче не будем добавлять стопслова

Будем использовать TfidfVectorizer с min_df = 2 
и попробуем добавить би-граммы

In [70]:
pipeline_words_1 = Pipeline([("vectorizer", TfidfVectorizer(min_df=2, ngram_range=(1,2), analyzer='word')), ("Regression", LinearSVC())])
print cross_val_score(pipeline_words_1, train.Text, train.Target, scoring='accuracy', cv=5).mean()

0.781490696816855


n-граммы

In [72]:
pipeline_words_1 = Pipeline([("vectorizer", TfidfVectorizer(min_df=2, ngram_range=(3,5), analyzer='char_wb')), ("Regression", LinearSVC())])
print cross_val_score(pipeline_words_1, train.Text, train.Target, scoring='accuracy', cv=5).mean()

0.780991937449609


# Test

In [96]:
pipeline = Pipeline([("vectorizer", TfidfVectorizer(min_df=2, ngram_range=(1,2), analyzer='word')), ("Regression", LinearSVC())])
pipeline.fit(train.Text, train.Target)

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=2,
        ngram_range=(1, 2), norm=u'l2', preprocessor=None, smooth_idf...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [137]:
y = pd.DataFrame(range(0,len(pipeline.predict(test.Text[1:]))), columns=['Id'])
y['y'] = pipeline.predict(test.Text[1:])

# Answer

In [139]:
y.to_csv('out.csv', index = False)